In [7]:
from typing_extensions import TypedDict
from typing import List
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

llm = init_chat_model("openai:gpt-4o")

In [8]:
class State(TypedDict):

    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str


class Ingredient(BaseModel):

    name: str
    quantity: str
    unit: str


class IngredientsOutput(BaseModel):

    ingredients: List[Ingredient]

In [9]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    response = structured_llm.invoke(
        f"List 5-8 ingredients needed to make {state['dish']}"
    )
    return {
        "ingredients": response.ingredients,
    }


def create_recipe(state: State):
    response = llm.invoke(
        f"Write a step by step cooking instruction for {state["dish"]}, using these ingredients {state['ingredients']}",
    )
    return {
        "recipe_steps": response.content,
    }


def describe_plating(state: State):
    response = llm.invoke(
        f"Describe how to beautifully plate this dish {state["dish"]} based on this recipe {state["recipe_steps"]}"
    )
    return {
        "plating_instructions": response.content,
    }

In [10]:
graph_builder = StateGraph(State)

graph_builder.add_node("list_ingredients", list_ingredients)
graph_builder.add_node("create_recipe", create_recipe)
graph_builder.add_node("describe_plating", describe_plating)

graph_builder.add_edge(START, "list_ingredients")
graph_builder.add_edge("list_ingredients", "create_recipe")
graph_builder.add_edge("create_recipe", "describe_plating")
graph_builder.add_edge("describe_plating", END)

graph = graph_builder.compile()

In [11]:
graph.invoke({"dish": "hummus"})

{'dish': 'hummus',
 'ingredients': [Ingredient(name='Chickpeas', quantity='1', unit='cup'),
  Ingredient(name='Tahini', quantity='1/4', unit='cup'),
  Ingredient(name='Olive Oil', quantity='2', unit='tablespoons'),
  Ingredient(name='Lemon Juice', quantity='2', unit='tablespoons'),
  Ingredient(name='Garlic', quantity='1-2', unit='cloves'),
  Ingredient(name='Salt', quantity='1/2', unit='teaspoon'),
  Ingredient(name='Cumin', quantity='1/2', unit='teaspoon'),
  Ingredient(name='Water', quantity='2-4', unit='tablespoons')],
 'recipe_steps': "Here's a step-by-step guide to preparing delicious hummus using the provided ingredients:\n\n**Ingredients:**\n- Chickpeas: 1 cup (cooked or canned)\n- Tahini: 1/4 cup\n- Olive Oil: 2 tablespoons\n- Lemon Juice: 2 tablespoons\n- Garlic: 1-2 cloves\n- Salt: 1/2 teaspoon\n- Cumin: 1/2 teaspoon\n- Water: 2-4 tablespoons\n\n**Instructions:**\n\n1. **Prepare the Chickpeas:**\n   - If using canned chickpeas, drain and rinse them thoroughly under cold wate